In [1]:
# Create server parameters for stdio connection
from langchain_core.messages import AIMessage, HumanMessage, ToolCall
from langchain_mcp_adapters.tools import load_mcp_tools
from langgraph.prebuilt import create_react_agent
from dotenv import load_dotenv
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent

In [2]:
load_dotenv()

True

In [3]:
# Layer 2: Middleware between MCP Servers and Agent
client = MultiServerMCPClient(
    {
        "math": {
            "command": "python",
            # Make sure to update to the full absolute path to your math_server.py file
            "args": ["./math_server.py"],
            "transport": "stdio",
        },
        # Add more MCP Servers here...
    }
)

In [4]:
tools = await client.get_tools()
agent = create_react_agent("openai:gpt-4.1", tools)
math_response = await agent.ainvoke({"messages": "what's (3 + 5) x 12?"})

In [5]:
def parse_agent_response(response):
    for message in response['messages']:
        if isinstance(message, HumanMessage):
            print(f'Human: {message.content}')
        if isinstance(message, AIMessage):
            if message.tool_calls:
                print(f'Tool Call: {message.tool_calls}')
            else:
                print(f'Agent: {message.content}')

In [6]:
parse_agent_response(math_response)

Human: what's (3 + 5) x 12?
Tool Call: [{'name': 'add', 'args': {'a': 3, 'b': 5}, 'id': 'call_bHnJCosEr6tdxHve1QxSkBff', 'type': 'tool_call'}]
Tool Call: [{'name': 'multiply', 'args': {'a': 8, 'b': 12}, 'id': 'call_h3DSXYBb08Ys5bQTz1tXTDee', 'type': 'tool_call'}]
Agent: (3 + 5) × 12 = 8 × 12 = 96.


In [7]:
client = MultiServerMCPClient(
    {
        "math": {
            "command": "python",
            # Make sure to update to the full absolute path to your math_server.py file
            "args": ["./math_server.py"],
            "transport": "stdio",
        },
        "weather": {
            "command": "python",
            # Make sure to update to the full absolute path to your math_server.py file
            "args": ["./weather_server.py"],
            "transport": "stdio",
        },
        # Add more MCP Servers here...
    }
)

In [10]:
tools = await client.get_tools()
agent = create_react_agent("openai:gpt-4.1", tools)
agent_response = await agent.ainvoke({"messages": "How's the weather in San Francisco"})

In [11]:
parse_agent_response(agent_response)

Human: How's the weather in San Francisco
Tool Call: [{'name': 'get_weather_status', 'args': {'location': 'San Francisco'}, 'id': 'call_aeKxBFxNb7UNAQjd6BNFT1ZS', 'type': 'tool_call'}]
Agent: It's very sunny in San Francisco right now. If you need a detailed forecast or tips for enjoying the sunshine, let me know!
